In [1]:
#Import the required Libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def getCategoricalValues(df):
    cat_columns = df.select_dtypes(include=['object']).columns.tolist()
    cat_columns.sort()
    print(f"Columns with categorical values {len(cat_columns)} --> {cat_columns}")
    for col in cat_columns:
        count_values = df[col].unique().shape[0]
        print(f"{col:27} : [{count_values:2}] --> {sorted((df[col].unique().tolist())) if np.NaN not in df[col].unique().tolist() else (df[col].unique().tolist())}") 

In [9]:
def getNumCategoricalValues(df, ncat_columns):
    ncat_columns.sort()
    print(f"Columns with categorical values {len(ncat_columns)}")
    for col in ncat_columns:
        count_values = df[col].unique().shape[0]
        if count_values > 30:
            print(f"{col:26} : [{count_values:4}] --> {sorted(df[col].unique().tolist()[:10])}...{sorted(df[col].unique().tolist()[-10:])}") 
        else:
            print(f"{col:26} : [{count_values:4}] --> {sorted(df[col].unique().tolist())}") 

## Data Exploration 

### Data Reading & Data Types 

In [5]:
#Read the data in pandas
prev_app_df = pd.read_csv("data/previous_application.csv")
curr_app_df = pd.read_csv("data/application_data.csv")

In [6]:
prev_app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [36]:
PA_COLUMNS = prev_app_df.columns.tolist()
PA_ID_COLUMNS = ['SK_ID_PREV','SK_ID_CURR']
PA_NCAT_COLUMNS = ['NFLAG_INSURED_ON_APPROVAL', 'NFLAG_LAST_APPL_IN_DAY', 'SELLERPLACE_AREA', 'HOUR_APPR_PROCESS_START']
print(len(max(PA_COLUMNS)))
getNumCategoricalValues(prev_app_df, PA_NCAT_COLUMNS)

26
Columns with categorical values 4
HOUR_APPR_PROCESS_START    : [  24] --> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
NFLAG_INSURED_ON_APPROVAL  : [   3] --> [0.0, 1.0, nan]
NFLAG_LAST_APPL_IN_DAY     : [   2] --> [0, 1]
SELLERPLACE_AREA           : [2097] --> [-1, 6, 20, 35, 50, 61, 83, 130, 136, 200]...[887, 1164, 1419, 1948, 2054, 2164, 2233, 2420, 3420, 6989]


In [11]:
getCategoricalValues(prev_app_df)

Columns with categorical values 16 --> ['CHANNEL_TYPE', 'CODE_REJECT_REASON', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_CLIENT_TYPE', 'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_PAYMENT_TYPE', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'NAME_SELLER_INDUSTRY', 'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION', 'WEEKDAY_APPR_PROCESS_START']
CHANNEL_TYPE                : [ 8] --> ['AP+ (Cash loan)', 'Car dealer', 'Channel of corporate sales', 'Contact center', 'Country-wide', 'Credit and cash offices', 'Regional / Local', 'Stone']
CODE_REJECT_REASON          : [ 9] --> ['CLIENT', 'HC', 'LIMIT', 'SCO', 'SCOFR', 'SYSTEM', 'VERIF', 'XAP', 'XNA']
FLAG_LAST_APPL_PER_CONTRACT : [ 2] --> ['N', 'Y']
NAME_CASH_LOAN_PURPOSE      : [25] --> ['Building a house or an annex', 'Business development', 'Buying a garage', 'Buying a holiday home / land', 'Buying a home', 'Buying a new car', 'Buying a used car', 'Car repairs', 'Education', 'Everyday 

In [16]:
prev_app_df[prev_app_df.columns.difference(PA_ID_COLUMNS + PA_NCAT_COLUMNS)].describe()

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CNT_PAYMENT,DAYS_DECISION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_TERMINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED
count,1.297979e+06,1.670214e+06,1.670213e+06,7.743700e+05,1.284699e+06,1.297984e+06,1.670214e+06,997149.000000,997149.000000,997149.000000,997149.000000,997149.000000,774370.000000,5951.000000,5951.000000
mean,1.595512e+04,1.752339e+05,1.961140e+05,6.697402e+03,2.278473e+05,1.605408e+01,-8.806797e+02,342209.855039,13826.269337,76582.403064,33767.774054,81992.343838,0.079637,0.188357,0.773503
std,1.478214e+04,2.927798e+05,3.185746e+05,2.092150e+04,3.153966e+05,1.456729e+01,7.790997e+02,88916.115833,72444.869708,149647.415123,106857.034789,153303.516729,0.107823,0.087671,0.100879
min,0.000000e+00,0.000000e+00,0.000000e+00,-9.000000e-01,0.000000e+00,0.000000e+00,-2.922000e+03,-2922.000000,-2892.000000,-2889.000000,-2801.000000,-2874.000000,-0.000015,0.034781,0.373150
25%,6.321780e+03,1.872000e+04,2.416050e+04,0.000000e+00,5.084100e+04,6.000000e+00,-1.300000e+03,365243.000000,-1628.000000,-1314.000000,-1242.000000,-1270.000000,0.000000,0.160716,0.715645
50%,1.125000e+04,7.104600e+04,8.054100e+04,1.638000e+03,1.123200e+05,1.200000e+01,-5.810000e+02,365243.000000,-831.000000,-537.000000,-361.000000,-499.000000,0.051605,0.189122,0.835095
75%,2.065842e+04,1.803600e+05,2.164185e+05,7.740000e+03,2.340000e+05,2.400000e+01,-2.800000e+02,365243.000000,-411.000000,-74.000000,129.000000,-44.000000,0.108909,0.193330,0.852537
max,4.180581e+05,6.905160e+06,6.905160e+06,3.060045e+06,6.905160e+06,8.400000e+01,-1.000000e+00,365243.000000,365243.000000,365243.000000,365243.000000,365243.000000,1.000000,1.000000,1.000000


In [17]:
prev_app_df.loc[prev_app_df['NAME_PRODUCT_TYPE']!='x-sell'][['NAME_PRODUCT_TYPE','NAME_SELLER_INDUSTRY']]

,NAME_PRODUCT_TYPE,NAME_SELLER_INDUSTRY
0,XNA,Connectivity
4,walk-in,XNA
6,XNA,XNA
7,XNA,XNA
8,XNA,XNA
...,...,...
1670206,walk-in,XNA
1670207,walk-in,XNA
1670209,XNA,Furniture
1670210,XNA,Furniture


In [18]:
prev_app_df[['NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE','RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION']]

,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_YIELD_GROUP,PRODUCT_COMBINATION
0,POS,XNA,0.000000,0.182832,0.867336,middle,POS mobile with interest
1,Cash,x-sell,NaN,NaN,NaN,low_action,Cash X-Sell: low
2,Cash,x-sell,NaN,NaN,NaN,high,Cash X-Sell: high
3,Cash,x-sell,NaN,NaN,NaN,middle,Cash X-Sell: middle
4,Cash,walk-in,NaN,NaN,NaN,high,Cash Street: high
...,...,...,...,...,...,...,...
1670209,POS,XNA,0.000000,NaN,NaN,low_normal,POS industry with interest
1670210,POS,XNA,0.340554,NaN,NaN,middle,POS industry with interest
1670211,POS,XNA,0.101401,NaN,NaN,low_normal,POS household with interest
1670212,Cash,x-sell,NaN,NaN,NaN,low_normal,Cash X-Sell: low


- Understand domain, each column
- Column Vs Missing values

There could be multiple applications from the same customer
Amounts -> Can they be rounded off?
Goods Vs Amount loan

## Application Data

In [19]:
curr_app_df = pd.read_csv("data/application_data.csv")

In [35]:
curr_app_df.iloc[:,:58].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 58 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int64  
 1   TARGET                       307511 non-null  int64  
 2   NAME_CONTRACT_TYPE           307511 non-null  object 
 3   CODE_GENDER                  307511 non-null  object 
 4   FLAG_OWN_CAR                 307511 non-null  object 
 5   FLAG_OWN_REALTY              307511 non-null  object 
 6   CNT_CHILDREN                 307511 non-null  int64  
 7   AMT_INCOME_TOTAL             307511 non-null  float64
 8   AMT_CREDIT                   307511 non-null  float64
 9   AMT_ANNUITY                  307499 non-null  float64
 10  AMT_GOODS_PRICE              307233 non-null  float64
 11  NAME_TYPE_SUITE              306219 non-null  object 
 12  NAME_INCOME_TYPE             307511 non-null  object 
 13 

In [33]:
curr_app_df.iloc[:,58:].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 64 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   APARTMENTS_MODE               151450 non-null  float64
 1   BASEMENTAREA_MODE             127568 non-null  float64
 2   YEARS_BEGINEXPLUATATION_MODE  157504 non-null  float64
 3   YEARS_BUILD_MODE              103023 non-null  float64
 4   COMMONAREA_MODE               92646 non-null   float64
 5   ELEVATORS_MODE                143620 non-null  float64
 6   ENTRANCES_MODE                152683 non-null  float64
 7   FLOORSMAX_MODE                154491 non-null  float64
 8   FLOORSMIN_MODE                98869 non-null   float64
 9   LANDAREA_MODE                 124921 non-null  float64
 10  LIVINGAPARTMENTS_MODE         97312 non-null   float64
 11  LIVINGAREA_MODE               153161 non-null  float64
 12  NONLIVINGAPARTMENTS_MODE      93997 non-null

In [21]:
print(len(max(curr_app_df.columns.tolist())))

16


In [44]:
CA_COLUMNS = curr_app_df.columns.tolist()
CA_ID_COLUMNS = ['SK_ID_CURR']
CA_NCAT_COLUMNS = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY','FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL',
                    'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION',
                    'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY',
                    'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 
                    'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13','FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 
                    'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21']
print(len(max(CA_COLUMNS)))
getNumCategoricalValues(curr_app_df, CA_NCAT_COLUMNS)

16
Columns with categorical values 36
FLAG_CONT_MOBILE           : [   2] --> [0, 1]
FLAG_DOCUMENT_10           : [   2] --> [0, 1]
FLAG_DOCUMENT_11           : [   2] --> [0, 1]
FLAG_DOCUMENT_12           : [   2] --> [0, 1]
FLAG_DOCUMENT_13           : [   2] --> [0, 1]
FLAG_DOCUMENT_14           : [   2] --> [0, 1]
FLAG_DOCUMENT_15           : [   2] --> [0, 1]
FLAG_DOCUMENT_16           : [   2] --> [0, 1]
FLAG_DOCUMENT_17           : [   2] --> [0, 1]
FLAG_DOCUMENT_18           : [   2] --> [0, 1]
FLAG_DOCUMENT_19           : [   2] --> [0, 1]
FLAG_DOCUMENT_2            : [   2] --> [0, 1]
FLAG_DOCUMENT_20           : [   2] --> [0, 1]
FLAG_DOCUMENT_21           : [   2] --> [0, 1]
FLAG_DOCUMENT_3            : [   2] --> [0, 1]
FLAG_DOCUMENT_4            : [   2] --> [0, 1]
FLAG_DOCUMENT_5            : [   2] --> [0, 1]
FLAG_DOCUMENT_6            : [   2] --> [0, 1]
FLAG_DOCUMENT_7            : [   2] --> [0, 1]
FLAG_DOCUMENT_8            : [   2] --> [0, 1]
FLAG_DOCUMENT_9       

In [43]:
# Common columns between the 2 datasets
print(set(PA_COLUMNS) & set(CA_COLUMNS))

{'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'SK_ID_CURR', 'HOUR_APPR_PROCESS_START', 'AMT_CREDIT', 'NAME_CONTRACT_TYPE'}


In [22]:
getCategoricalValues(curr_app_df)

Columns with categorical values 16 --> ['CODE_GENDER', 'EMERGENCYSTATE_MODE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'NAME_TYPE_SUITE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'WALLSMATERIAL_MODE', 'WEEKDAY_APPR_PROCESS_START']
CODE_GENDER                 : [ 3] --> ['F', 'M', 'XNA']
EMERGENCYSTATE_MODE         : [ 3] --> ['No', nan, 'Yes']
FLAG_OWN_CAR                : [ 2] --> ['N', 'Y']
FLAG_OWN_REALTY             : [ 2] --> ['N', 'Y']
FONDKAPREMONT_MODE          : [ 5] --> ['reg oper account', nan, 'org spec account', 'reg oper spec account', 'not specified']
HOUSETYPE_MODE              : [ 4] --> ['block of flats', nan, 'terraced house', 'specific housing']
NAME_CONTRACT_TYPE          : [ 2] --> ['Cash loans', 'Revolving loans']
NAME_EDUCATION_TYPE         : [ 5] --> ['Academic degree', 'Higher education', 'Incomplete higher', 'Lower secon

In [ ]:
prev_app_df[curr_app_df.columns.difference(['SK_ID_PREV','SK_ID_CURR','NFLAG_INSURED_ON_APPROVAL', 'NFLAG_LAST_APPL_IN_DAY', 'SELLERPLACE_AREA', 'HOUR_APPR_PROCESS_START'])].describe()